In [1]:
# Importing the neccessary libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow
import tensorflow.keras as keras
from PIL import Image

In [2]:
# Loading the data

# Data consists of over 50000 images of traffic signs belonging to 43 diffrent classes
# The train folder has over 39000 images for training and Test folder contains of 12000 images.
# The Meta folder contains a image of each class
# The CSV file contains of image dimension,its path and corresponding label


cwd="../input/gtsrb-german-traffic-sign/"
os.listdir(cwd)

In [3]:
meta = os.listdir(cwd+'Meta')
meta.remove('.~lock.ClassesInformation.ods#')
meta.remove('.~lock.ClassesInformationStrong.ods#')
meta

In [4]:
# Viewing images belonging to each class

plt.figure(figsize=(30,30))
for i, file in enumerate(meta):
        img = Image.open(cwd+'Meta/'+file)
        ax = plt.subplot(9,5, i+1)
        ax.imshow(img)
        ax.set_title(file, size=20)
        plt.axis("off")
plt.tight_layout()

In [5]:
# creating dictionary for classlabels

classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [6]:
train_df = pd.read_csv(cwd+"Train.csv")
train_df.head()

In [7]:
train_df.describe()

In [8]:
# The average dimensions of all images are 50*50 pixels
# All images are resized as 50,50 pixels and saved as numpy array
# The data is normalised and the labels are one hot encoded

print(f'minimum width: {train_df.Width.min()}')
print(f'minimum height: {train_df.Height.min()}')
print(f'average width: {train_df.Width.mean()}')
print(f'average height: {train_df.Height.mean()}')

In [9]:
train_x =[]
for i in train_df.Path:
    img = Image.open(cwd+i)            # reading image
    img = img.resize((50,50))          # resizing image
    train_x.append(np.array(img))      # saving image as array to train
    
train_y = np.array(train_df.ClassId)
train_x = np.array(train_x)
print(train_x.shape)
print(train_y.shape)

In [10]:
test_df = pd.read_csv(cwd+'Test.csv')
test_df.head()

In [11]:
test_df.describe()

In [12]:
test_x=[]
for i in test_df.Path:
    img = Image.open(cwd+i)
    img = img.resize((50,50))
    test_x.append(np.array(img))
    
test_y = np.array(test_df.ClassId)
test_x = np.array(test_x)
print(test_x.shape)
print(test_y.shape)

In [13]:
print(classes[train_y[2]])
plt.imshow(train_x[2])
plt.axis('off')
plt.show()

In [14]:
# Normalising the Data

xtrain = train_x/255
xtest  = test_x/255

# one hot encoding

ytrain = keras.utils.to_categorical(train_y)
ytest = keras.utils.to_categorical(test_y)

In [15]:
# Building CNN model

from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size=0.2,random_state= 0)
print(xtrain.shape)
print(xvalid.shape)
print(ytrain.shape)
print(yvalid.shape)

In [16]:
import keras
import tensorflow

In [17]:
from keras import Sequential
from keras.layers import Conv2D,MaxPool2D,BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Activation

cnn=Sequential()


cnn.add(Conv2D(filters=32, kernel_size= (5,5), strides=2, activation='relu', padding='same', input_shape=(50,50,3)))
cnn.add(Conv2D(filters=32, kernel_size=(5,5), strides=2, activation='relu', padding='same'))
cnn.add(MaxPool2D((2,2), padding='valid'))
cnn.add(Dropout(0.2))



cnn.add(Conv2D(filters=64, kernel_size =(5,5), strides=2, activation='relu',  padding='same'))
cnn.add(Conv2D(filters=64, kernel_size =(5,5), strides=2, activation='relu',  padding='same'))
cnn.add(MaxPool2D((2,2), padding='valid'))
cnn.add(Dropout(0.2))

cnn.add(Flatten())
cnn.add(Dense(300, activation='relu'))
cnn.add(Dense(200, activation='relu'))
cnn.add(Dense(43, activation='softmax'))

cnn.summary()

In [18]:
# Compiling the CNN model

cnn.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])

In [19]:
# fitting the model with early call back

earlystop_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights = True)

cnn_history = cnn.fit(xtrain, ytrain, epochs=200, validation_data=(xvalid,yvalid),callbacks=[earlystop_cb])

In [20]:
pd.DataFrame(cnn_history.history).plot

In [21]:
## Evaluation of CNN

loss, accuracy = cnn.evaluate(xtest,ytest)
print(f'loss={loss:.2f}\naccuaracy={accuracy*100:.2f}%')

In [22]:
# Prediction

prob = cnn.predict(xtest)
pred = np.argmax(prob, axis=-1)

In [23]:
# Print the predicted images


plt.figure(figsize=(30,30))
for i in range(50):
    ax = plt.subplot(5,10, i + 1)
    ax.imshow(xtest[i])
    ax.set_title(f'predicted:{classes[pred[i]]}\nactual:{classes[test_y[i]]}')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [24]:
# Applying Confusion matrix to find of the accuracy

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

cm = confusion_matrix(pred,test_y)

print(cm)
print(classification_report(pred,test_y))

In [32]:
import seaborn as sns
df_cnn = pd.DataFrame(cm, index = classes, columns= classes)
plt.figure(figsize=(20,20))
sns.heatmap(df_cnn, annot=True)

In [25]:
# Accuracy of the prediction is 0.93%